# Preprocessing pipeline


This pipeline aims to serve as a semiautomatic and reproducible framework for preprocessing EEG signals before performing time-frequency-based analysis. It minimizes the manual steps required to clean the data based on visual inspection. It is advised to revisit the cleaned epochs before writing the final preprocessed file. 


## Outline

1. __Temporal filtering__

High-frequency artefacts and slow drifts are removed with a zero-phase bandpass filter using mne-Python [1]. The cutoff frequencies (0.5 - 45 Hz) can be modified in the utils folder in the configuration file (config.py). 


2. __Create epochs__ 

Epochs are nonoverlapping data segments created from the continuous data with a duration of 1 seconds. The length of epochs can be changed in the configuration file.
Epochs can be created from (1) events; there is a custom method that created epochs based on annotations in the raw data, (2) without events, data segments are created from the beginning of the raw data. 


3. __Outlier data rejection__  

    3.1. _Preliminar rejection_  
Epochs are rejected based on a global threshold on the z-score (> 3) of the epoch variance and amplitude range.

    3.2. _ICA decomposition_  
The default method is the infomax algorithm, however it can be changed in the configuration file along with the number of components and the decimation parameter. Components containing blink artefacts are automatically marked with mne-Python.
The ICA sourced can be visualized and interactively selected and rejected based on their topographies, time-courses or frequency spectra. The number of components that were removed from the data are documented in the “description” field of the epochs instance “info” structure.

    3.3. _Autoreject_  
Autoreject [2, 3] uses unsupervised learning to estimate the rejection threshold for the epochs. In order to reduce computation time that increases with the number of segments and channels, autoreject can be fitted on a representative subset of epochs (25% of total epochs). Once the parameters are learned, the solution can be applied to any data that contains channels that were used during fit.


4. __Outlier channel interpolation__

The Random Sample Consensus (RANSAC) algorithm [4] selects a random subsample of good channels to make predictions of each channel in small non-overlapping 4 seconds long time windows. It uses a method of spherical splines (Perrin et al., 1989) to interpolate the bad sensors. The sensors that were interpolated are added to the "description" field of the epochs "info" structure. 


<img src="static/preprocessing_pipeline_diagram.svg">


## References

[1] A. Gramfort, M. Luessi, E. Larson, D. Engemann, D. Strohmeier, C. Brodbeck, R. Goj, M. Jas, T. Brooks, L. Parkkonen, M. Hämäläinen, MEG and EEG data analysis with MNE-Python, Frontiers in Neuroscience, Volume 7, 2013, ISSN 1662-453X

[2] Mainak Jas, Denis Engemann, Federico Raimondo, Yousra Bekhti, and Alexandre Gramfort, “Automated rejection and repair of bad trials in MEG/EEG.” In 6th International Workshop on Pattern Recognition in Neuroimaging (PRNI), 2016.

[3] Mainak Jas, Denis Engemann, Yousra Bekhti, Federico Raimondo, and Alexandre Gramfort. 2017. “Autoreject: Automated artifact rejection for MEG and EEG data”. NeuroImage, 159, 417-429.

[4] Bigdely-Shamlo, N., Mullen, T., Kothe, C., Su, K. M., & Robbins, K. A. (2015). The PREP pipeline: standardized preprocessing for large-scale EEG analysis. Frontiers in neuroinformatics, 9, 16.



## Import packages


```%matplotlib qt``` is the recommended backend for interactive visualization (can be slower);    

switch to ```%matplotlib inline``` for (faster) static plots

In [1]:
import os
from ipyfilechooser import FileChooser

from meeg_tools.preprocessing import *
from meeg_tools.utils.epochs import create_epochs_from_events, create_metadata
from meeg_tools.utils.raw import read_raw_measurement, filter_raw
from meeg_tools.utils.log import update_log

from meeg_tools.utils.config import settings

from matplotlib import pyplot as plt
%matplotlib qt

## Load raw data


See [this](https://mne.tools/stable/auto_tutorials/io/20_reading_eeg_data.html) documentation for help with supported file formats.  

In [ ]:
# Use the widget to navigate to the experiment folder path and select an EEG file 
base_path = '/Volumes/crnl-memo-hd/TMS_rewiring/'
fc = FileChooser(base_path)
fc.filter_pattern = ['*.vhdr', '*.edf']

display(fc)

In [ ]:
# Load selected file
raw = read_raw_measurement(raw_file_path=fc.selected)
raw.info

## Temporal filtering

- bandpass filter (0.5 - 45 Hz)

In [ ]:
settings['bandpass_filter']

In [ ]:
raw_bandpass = filter_raw(raw)

## Create epochs

- select the events for analysis

In [ ]:
settings['epochs']['start_time'] = -0.25
settings['epochs']['end_time'] = 0.75

In [ ]:
events_ids = np.concatenate([np.arange(10, 53, 1), 
                             np.arange(10, 53, 1) + 100,
                            [211, 212, 213, 214, 215, 216]])

epochs = create_epochs_from_events(raw=raw_bandpass, event_ids=[events_ids])

## Create metadata for epochs (optional)

- adding metadata makes it easier to select epochs of different types
- custom triggers are selected from the raw instance

- metadata can be added or replaced later (e.g. after preprocessing)

In [ ]:
metadata = create_metadata(epochs)
metadata.head(10)

In [ ]:
epochs.metadata = metadata

In [ ]:
# we can filter epochs as a query
epochs["triplet == 'H' & answer == 'incorrect'"]

In [ ]:
epochs["epoch == 5 & triplet == 'L'"]

In [ ]:
# subselecting epochs 
# Here we could also include thrills, repetitions, or practice stimuli.
# ICA should not run on duplicate data (epochs should not be overlapping!)

epochs = epochs["triplet == 'L' | triplet == 'H'"]

## Run preprocessing


### 1.1. Preliminary epoch rejection

In [ ]:
epochs_faster = prepare_epochs_for_ica(epochs=epochs)

### 1.2. Run ICA


The parameters are: 32 ICA components using ["infomax"](https://mne.tools/stable/generated/mne.preprocessing.infomax.html) algorithm. 

When visualizing the components, it is recommended to subset the data (see below).

In [ ]:
ica = run_ica(epochs=epochs_faster)

In [ ]:
# Plot component topographies
ica.plot_components()

In [ ]:
# Visualize components on epochs
# Subset epochs to reduce execution time (e.g. take epochs from every 5th event)
subset = list(epochs.event_id.keys())[::5]
# Exclude components by selecting them, right click on component name to visulize source:
ica.plot_sources(epochs_faster[subset])

In [ ]:
# After selecting the components to exclude, apply ICA to epochs
# Document the number of excluded components
ica.apply(epochs_faster.load_data())
epochs_faster.info['description'] = f'n_components: {len(ica.exclude)}'

### 1.3. Visualize ICA cleaned epochs (optional)

This step can be repeated after each preprocessing step, or you can also do a final inspection at the end. 

In [ ]:
# Optional
epochs_faster[subset].plot(n_epochs=10, n_channels=32, scalings={'eeg': 20e-6},)


In [ ]:
# Optional
# If you found a component that should have been excluded but it wasn't you can exclude it here:
ica.plot_sources(epochs_faster)

# After selecting the components to exclude, apply ICA to epochs
# Document the number of excluded components
ica.apply(epochs_faster)
epochs_faster.info['description'] = f'n_components: {len(ica.exclude)}'

### 1.4. Save cleaned epochs (recommended)

In [ ]:
# Create folder for preprocessed and interim files
folder_name = 'preprocessed'
epochs_path = os.path.join(base_path, folder_name, 'epochs_asrt')


# Create path to epoch files
if not os.path.exists(epochs_path):
    os.makedirs(epochs_path)

# Save ICA cleaned epochs 
fid = epochs.info['fid']
epochs_faster.info['fid'].update(f'{fid}_ICA')
postfix = '-epo.fif.gz'
epochs_faster.save(os.path.join(epochs_path, f'{epochs_faster.info['fid']}{postfix}'), overwrite=True)

In [ ]:
epochs_faster.info

### 1.5. Create a log file 

We can create a log file for the preprocessed data and store metadata
that could be useful to remember. You can add more columns to this, or 
remove the ones that are not needed. For documentation purporses, it is 
recommended to store the number of rejected and total epochs, the number of
ICA components that were rejected, the number of interpolated electrodes etc.
You can also add a column with "notes" to add custom descriptions about the data.

In [ ]:
log_file_path = os.path.join(os.path.join(base_path, 'preprocessed', 'epochs_asrt'), 'log.csv')
update_log(log_file_path, epochs_faster, '')

In [ ]:
log

### 2.1. Run autoreject

In [ ]:
reject_log = run_autoreject(epochs_faster.load_data(), n_jobs=11, subset=True)

In [ ]:
# Here you can decide how strict should be the epoch rejection.
# You can drop only those that were marked as bad epochs, or a more 
# strict rejection threshold can be if you drop epochs where more than
# 15% of the channels were marked as noisy.

# You can plot the epochs with Autoreject, where bad epochs are marked with
# red colors. 

# reject_log.plot_epochs(epochs_faster)


# rejecting only bad epochs
# epochs_autoreject = epochs_faster.copy().drop(reject_log.bad_epochs, reason='AUTOREJECT')

# rejecting those epochs too where more than 15% of the channels are marked as noisy
bads = np.where(np.count_nonzero(reject_log.labels, axis=1) > 0.15 * epochs_faster.info['nchan'])[0].tolist()

# you can plot just the bad epochs to double check how strict this rejection is
# if bads: 
#     epochs_faster[bads].plot(n_epochs=10,
#                                 scalings={'eeg': 20e-6},
#                                 n_channels=32)


epochs_autoreject = epochs_faster.copy().drop(bads, reason='AUTOREJECT')

In [ ]:
epochs_autoreject.plot(n_epochs=10, n_channels=32, scalings={'eeg': 20e-6},)

In [ ]:
# save clean epochs
fid = epochs.info['fid']
epochs_autoreject.info['fid'].update(f'{fid}_ICA_autoreject')
postfix = '-epo.fif.gz'
epochs_autoreject.save(os.path.join(epochs_path, f'{epochs_autoreject.info['fid']}{postfix}'), overwrite=True)

In [ ]:
update_log(log_file_path, epochs_autoreject, '')

### 3. Run ransac

In [ ]:
ransac = run_ransac(epochs)

In [ ]:
epochs_ransac = epochs_autoreject.copy()
epochs_ransac.info['bads'] = ransac.bad_chs_
# Alternatively, you can interpolate the channels that were reported before 
# epochs_ransac.info['bads'] = ransac.report

epochs_ransac.interpolate_bads(reset_bads=True)

In [ ]:
# inspect which sensors were interpolated (if any)
epochs_ransac.info

## 4. Final visual inspection

Mark epochs that should be dropped,  etc.

In [ ]:
epochs_ransac.plot(n_epochs=10,
                       n_channels=32,
                       # group_by='position',
                       scalings={'eeg': 20e-6})

In [ ]:
# If there were no bad channels found, however there are clearly some bad ones
# you can you the epochs_ransac object to mark bad channels

# example
# epochs_ransac.info['bads'] = ['Fp1', 'F7']
# epochs_ransac.interpolate_bads()
# bads_str = ', '.join(['Fp1', 'F7'])
# epochs_ransac.info.update(description=epochs_ransac.info['description'] + ', interpolated: ' + bads_str)
# log['n_interpolated'].update(len(['Fp1', 'F7']))


# However, if you found additional channels (addition to those that were found with RANSAC)
# then you should use the epochs_autoreject object (don't interpolate the same object twice)

# example
# epochs_ransac = epochs_autoreject.copy()
# epochs_ransac.info['bads'] = ['Fp1', 'F7']
# bads_str = ', '.join(['Fp1', 'F7'])
# epochs_ransac.info.update(description=epochs_ransac.info['description'] + ', interpolated: ' + bads_str)
# log['n_interpolated'].update(len(['Fp1', 'F7']))


## 5. Apply baseline and set average reference

### 5.1. Apply baseline (optional)



In [ ]:
epochs_ransac.apply_baseline(baseline=(-0.25, 0.0))

### 5.2. Set average reference

To set a “virtual reference” that is the average of all channels, you can use set_eeg_reference() with ref_channels='average'.

In [ ]:
epochs_ransac.set_eeg_reference('average')

## 6. Save cleaned epochs

In [ ]:
# save clean epochs
fid = epochs.info['fid']
epochs_ransac.info['fid'].update(f'{fid}_ICA_autoreject_ransac')
postfix = '-epo.fif.gz'
epochs_ransac.save(os.path.join(epochs_path, f'{epochs_ransac.info['fid']}{postfix}'), overwrite=True)

In [ ]:
update_log(log_file_path, epochs_ransac, '')

## Time-frequency analysis
### Evoked


In [ ]:
# Subset channels

ch_names = ['F7', 'F5', 'F3', 'FC5', 'FC3',
           'F1', 'Fz', 'F2', 'FC1', 'FCz', 'FC2',
           'F4', 'F6', 'F8', 'FC4', 'FC6',
           'FT7', 'T7', 'TP7', 
           'C3', 'Cz', 'C4',
           'FT8', 'T8', 'TP8',
           'CP5', 'CP3', 'P7', 'P5', 'P3',
           'CP1', 'CPz', 'CP2', 'P1', 'Pz', 'P2',
           'CP4', 'CP6', 'P4', 'P6', 'P8',
           'PO3', 'PO7', 'O1',
           'PO4', 'PO8', 'O2',]

epochs_evoked = epochs_ransac.copy().pick_channels(ch_names, ordered=True)

In [ ]:
fig = epochs_evoked.plot_sensors(show_names=True)

In [ ]:
epochs_evoked.plot(n_epochs=10,
                       n_channels=32,
                       # group_by='position',
                       scalings={'eeg': 20e-6})

In [ ]:
#bands = [(4, 8, 'Theta (4 - 8 Hz)'), (8, 13, 'Alpha (8 - 13 Hz)'), (13, 30, 'Beta (13 - 30 Hz)'), (30, 45, 'Gamma (30 - 45 Hz)')]
#epochs_ransac_ordered.plot_psd_topomap(bands=bands, vlim='joint', ch_type='eeg')

In [ ]:
evoked_epoch_1_H = epochs_evoked["epoch == 1 & triplet == 'H'"].average()

In [ ]:
evoked_epoch_1_H.detrend().plot()

In [ ]:
epochs_evoked.detrend().plot_joint(times=[0.0, 0.11, 0.15, 0.3, 0.6, 0.7])

In [ ]:
epochs_evoked["epoch == 1 & triplet == 'H'"].plot_psd()